In [236]:
# Installation de tous les modules nécessaires
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
!pip install lazypredict
from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

In [238]:
# Chargement et affichage du data frame
df = pd.read_csv(r'\Users\Lucie Montagne\Documents\DATASCIENTEST\PROJET\ADEME-CarLabelling-2023-local.csv', encoding='utf-8', sep=';')
df.head()

,Marque,Libellé modèle,Modèle,Groupe,Description Commerciale,Energie,Carrosserie,Cylindrée,Gamme,Puissance fiscale,...,Essai CO2 type 1,Essai HC,Essai Nox,Essai HCNox,Essai particules,Masse OM Min,Masse OM Max,Bonus-Malus,Barème Bonus-Malus,Prix véhicule
0,RENAULT,KANGOO,KANGOO,RENAULT,KANGOO (130ch),ESSENCE,COMBISPACE,1332,INFERIEURE,7,...,"0,234","0,012","0,022",NaN,"0,001",1594,1594,Malus,1901.00,31000
1,MAZDA,MX-30,MX-30,AUTRES,MX-30 e-SKYACTIV (145ch),ELECTRIC,TS TERRAINS/CHEMINS,0,INFERIEURE,6,...,NaN,NaN,NaN,NaN,NaN,1720,1720,Bonus 6000,-6000.00,38510
2,B.M.W.,SERIE 2,218,AUTRES,218i Active Tourer,ESSENCE,MONOSPACE COMPACT,1499,MOYENNE INFERIEURE,7,...,"0,204","0,007","0,009",NaN,NaN,1545,1545,Malus,400.00,36688
3,SKODA,OCTAVIA,OCTAVIA,VGF,OCTAVIA Berline 2.0 TDI (150ch) DSG7,GAZOLE,BERLINE,1968,MOYENNE SUPERIEURE,8,...,"0,026",NaN,"0,028","0,037",NaN,1461,1461,Neutre 0,NaN,35218
4,JEEP,COMPASS,COMPASS,STELLANTIS,1.3 Turbo T4 (240ch) BVA6 4xe Upland / Trailha...,ELEC+ESSENC HR,TS TERRAINS/CHEMINS,1332,MOYENNE SUPERIEURE,10,...,"0,326","0,033","0,019",NaN,NaN,1935,1935,Neutre 0,NaN,50100


# I. Features Engineering

In [240]:
# Renommage de certaines colonnes
df = df.rename(columns={'CO2 vitesse mixte Max': 'CO2_vitesse_mixte_Max'})
df = df.rename(columns={'Puissance fiscale': 'Puissance_fiscale'})
df = df.rename(columns={'Masse OM Max': 'Masse_OM_Max'})
df = df.rename(columns={'Type de boite': 'Type_de_boite'})

In [242]:
# Sélection des variables intéressantes pour notre problématique
df = df[['Energie', 'Carrosserie', 'Puissance_fiscale', 'Type_de_boite', 'CO2_vitesse_mixte_Max', 'Masse_OM_Max']]

In [244]:
# Suppression des lignes concernant les véhicules électriques (non concernés par la problématique)
df = df[df['Energie'] != 'ELECTRIC']

In [246]:
# Vérification de la prise en compte
df['Energie'].unique()

array(['ESSENCE', 'GAZOLE', 'ELEC+ESSENC HR', 'GAZ+ELEC HNR',
       'ESS+ELEC HNR', 'SUPERETHANOL', 'ESS+G.P.L.', 'ELEC+GAZOLE HR'],
      dtype=object)

In [248]:
# Affichage de la nouvelle dimension du jeu de données
print("Nouvelle dimension du jeu de données :", df.shape)

Nouvelle dimension du jeu de données : (3470, 6)


# II. Pré-processing

In [250]:
# Rappel du type des variables
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3470 entries, 0 to 3603
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Energie                3470 non-null   object
 1   Carrosserie            3470 non-null   object
 2   Puissance_fiscale      3470 non-null   int64 
 3   Type_de_boite          3470 non-null   object
 4   CO2_vitesse_mixte_Max  3470 non-null   object
 5   Masse_OM_Max           3470 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 189.8+ KB


In [252]:
# Changement du type de la variable 'CO2 vitesse mixte Max'
df['CO2_vitesse_mixte_Max'] = df['CO2_vitesse_mixte_Max'].astype(str).str.replace(',', '.').astype(float)

In [254]:
# Vérification de la prise en compte
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3470 entries, 0 to 3603
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Energie                3470 non-null   object 
 1   Carrosserie            3470 non-null   object 
 2   Puissance_fiscale      3470 non-null   int64  
 3   Type_de_boite          3470 non-null   object 
 4   CO2_vitesse_mixte_Max  3470 non-null   float64
 5   Masse_OM_Max           3470 non-null   int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 189.8+ KB


In [256]:
# Détection des doublons
duplicate = df.duplicated().sum()
print('Le nombre de doublons est de :', duplicate) #le nombre de doublons est de 2810

Le nombre de doublons est de : 2810


In [262]:
# Suppression des doublons
df = df.drop_duplicates(keep='first')

In [264]:
# Vérification de la prise en compte
duplicate = df.duplicated().sum()
print('Le nombre de doublons est de :', duplicate)

Le nombre de doublons est de : 0


In [290]:
# Séparation du jeu de données en 1 jeu d'entrainement et 1 jeu de test
feats = df.drop('CO2_vitesse_mixte_Max', axis=1)
target = df['CO2_vitesse_mixte_Max']

X_train, X_test, y_train, y_test = train_test_split(feats, target, test_size=0.20, random_state=42) #le jeu étant petit, nous devons avoir un entrainement élevé

In [282]:
import joblib
joblib.dump(X_train, 'X_train')

['X_train']

In [268]:
# Traitement des valeurs manquantes
df.isna().sum()

Energie                  0
Carrosserie              0
Puissance_fiscale        0
Type_de_boite            0
CO2_vitesse_mixte_Max    0
Masse_OM_Max             0
dtype: int64

In [270]:
# Attribution des colonnes numériques
num_train = X_train[['Puissance_fiscale', 'Masse_OM_Max']]
num_test = X_test[['Puissance_fiscale', 'Masse_OM_Max']]

# Attribution des colonnes catégorielles
cat_train = X_train[['Energie', 'Carrosserie', 'Type_de_boite']]
cat_test = X_test[['Energie', 'Carrosserie', 'Type_de_boite']]

In [272]:
# Encodage des variables catégorielles
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# utilisation du OneHotEncoder sans sparsité
oneh = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

# identification des colonnes catégorielles
cat = ['Energie', 'Carrosserie', 'Type_de_boite']

# encodage sur le jeu d'entraînement
cat_train_encoded = pd.DataFrame(oneh.fit_transform(X_train[cat]), columns=oneh.get_feature_names_out(cat), index=X_train.index)

# encodage sur le jeu de test
cat_test_encoded = pd.DataFrame(oneh.transform(X_test[cat]), columns=oneh.get_feature_names_out(cat), index=X_test.index)

# concaténation avec les autres colonnes non catégorielles
X_train = pd.concat([X_train.drop(cat, axis=1), cat_train_encoded], axis=1)
X_test = pd.concat([X_test.drop(cat, axis=1), cat_test_encoded], axis=1)

# vérification des nouvelles dimensions
print("Dimension du jeu d'entrainement :", X_train.shape)
print("Dimension du jeu de test :",X_test.shape)

Dimension du jeu d'entrainement : (528, 18)
Dimension du jeu de test : (132, 18)


In [274]:
# Mise à l'échelle des données numériques
sc = StandardScaler()
num_train_scaled = sc.fit_transform(num_train)
num_test_scaled = sc.transform(num_test)

# convertion en dataframe :
num_train_scaled = pd.DataFrame(num_train_scaled, columns=num_train.columns)
num_test_scaled = pd.DataFrame(num_test_scaled, columns=num_test.columns)

# conversion du résultat d'encodage en DataFrame sinon impossible de concaténer les données
oneh = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
cat_train_encoded = pd.DataFrame(oneh.fit_transform(cat_train), columns=oneh.get_feature_names_out(cat_train.columns))
cat_test_encoded = pd.DataFrame(oneh.transform(cat_test), columns=oneh.get_feature_names_out(cat_test.columns))

# reconstitution du jeux d'entraînement et de test (génération d'un nouvel index) :
X_train_concat = pd.concat([num_train_scaled.reset_index(drop=True), cat_train_encoded.reset_index(drop=True)], axis=1)
X_test_concat = pd.concat([num_test_scaled.reset_index(drop=True), cat_test_encoded.reset_index(drop=True)], axis=1)

# affichage des dimmensions :
print(X_train_concat.shape)
print(X_test_concat.shape)

(528, 18)
(132, 18)


In [284]:
joblib.dump(X_train_concat, "X_train_concat")

['X_train_concat']

# III. Test et choix des modèles

# *Les modèles les plus performants (ordre décroissant)*

In [ ]:
# Import du module LazyPredict
from lazypredict.Supervised import LazyRegressor, LazyClassifier

reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

print(models)

## a. Extra Trees Regressor

In [ ]:
# Importation de l'Extra Trees Regressor
from sklearn.ensemble import ExtraTreesRegressor

# instanciation du modèle Extra Trees Regressor et entrainement sur les données d'entraînement
et_regressor = ExtraTreesRegressor(random_state=42)
et_regressor.fit(X_train_concat, y_train)

In [ ]:
import joblib
joblib.dump(et_regressor, "model_et_regressor")

In [280]:
# évaluation du modèle à l'aide de la méthode score sur les données d'entraînement et de test (coefficient de détermination ou R2)
print("Score R2 sur les données d'entraînement :", et_regressor.score(X_train_concat, y_train))
print("Score R2 sur les données de test :", et_regressor.score(X_test_concat, y_test))

Score R2 sur les données d'entraînement : 0.9989545616198728
Score R2 sur les données de test : 0.9678605197212603


In [286]:
joblib.dump(et_regressor, "et_regressor")

['et_regressor']

In [278]:
# Calcul des autres métriques sur les jeux d'entrainement et de test (MAE, MSE, RMSE)
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_error
y_pred_et_train = et_regressor.predict(X_train_concat)
y_pred_et_test = et_regressor.predict(X_test_concat)

# jeu d'entraînement
mae_et_train = mean_absolute_error(y_train, y_pred_et_train)
mse_et_train = mean_squared_error(y_train, y_pred_et_train, squared=True)
rmse_et_train = mean_squared_error(y_train, y_pred_et_train, squared=False)

# jeu de test
mae_et_test = mean_absolute_error(y_test, y_pred_et_test)
mse_et_test = mean_squared_error(y_test, y_pred_et_test, squared=True)
rmse_et_test = mean_squared_error(y_test, y_pred_et_test, squared=False)

# création d'un DataFrame pour comparer les métriques des deux algorithmes
data = {
    'MAE train': [ mae_et_train],
    'MAE test': [mae_et_test],
    'MSE train': [mse_et_train],
    'MSE test': [mse_et_test],
    'RMSE train': [rmse_et_train],
    'RMSE test': [rmse_et_test]
}

df_metrics_et = pd.DataFrame(data, index=['Extra Trees'])

# affichage du DataFrame
df_metrics_et.head()

,MAE train,MAE test,MSE train,MSE test,RMSE train,RMSE test
Extra Trees,0.72,7.04,3.94,108.21,1.98,10.40


In [276]:
#Validation croisée sur le modèle Extra Trees Regressor
from sklearn.model_selection import cross_val_score
scores = cross_val_score(et_regressor, X_train_concat, y_train, cv=5, scoring='r2')
print("Scores de validation croisée : ", scores)
print("Moyenne des scores : ", scores.mean())

Scores de validation croisée :  [0.93045444 0.93286144 0.92818891 0.92000615 0.90502452]
Moyenne des scores :  0.923307091061502


In [ ]:
# Affichage de plusieurs arbres de l'Extra Trees Regressor
from sklearn.tree import plot_tree

n_trees_to_display = 3

for i, tree in enumerate(et_regressor.estimators_[:n_trees_to_display]):  # Limiter le nombre d'arbres affichés
    plt.figure(figsize=(20, 10))  # Ajuster la taille de la figure
    plot_tree(tree, filled=True)
    plt.title(f"Arbre {i+1} de l'Extra Trees Regressor")
    plt.show()

In [ ]:
# Nombre d'arbres utilisés dans l'Extra Trees Regressor
nombre_arbres = len(et_regressor.estimators_)
print(f"Nombre d'arbres générés par l'Extra Trees Regressor : {nombre_arbres}")

## b. XGB Regressor

In [ ]:
import xgboost as xgb

# Création d'une instance de XGBRegressor
xgb_regressor = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# entraînement du modèle
xgb_regressor.fit(X_train_concat, y_train)

In [ ]:
joblib.dump(xgb_regressor, "model_xgb_regressor")

In [288]:
# affichage du score R2
print(xgb_regressor.score(X_train_concat, y_train))
print(xgb_regressor.score(X_test_concat, y_test))

NotFittedError: need to call fit or load_model beforehand

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Prédictions
y_pred_xgb_train = xgb_regressor.predict(X_train_concat)
y_pred_xgb_test = xgb_regressor.predict(X_test_concat)


# jeu d'entraînement
mae_xgb_train = mean_absolute_error(y_train, y_pred_xgb_train)
mse_xgb_train = mean_squared_error(y_train, y_pred_xgb_train, squared=True)
rmse_xgb_train = mean_squared_error(y_train, y_pred_xgb_train, squared=False)


# jeu de test
mae_xgb_test = mean_absolute_error(y_test, y_pred_xgb_test)
mse_xgb_test = mean_squared_error(y_test, y_pred_xgb_test, squared=True)
rmse_xgb_test = mean_squared_error(y_test, y_pred_xgb_test, squared=False)

# Création d'un DataFrame pour comparer les métriques des trois algorithmes
data_xgb_test = {
    'MAE train': [mae_xgb_train],
    'MAE test': [mae_xgb_test],
    'MSE train': [mse_xgb_train],
    'MSE test': [mse_xgb_test],
    'RMSE train': [rmse_xgb_train],
    'RMSE test': [rmse_xgb_test]

}

df_xgb_test = pd.DataFrame(data_xgb_test, index=['XGBoost'])

# affichage du DataFrame
df_xgb_test.head()

In [ ]:
# Validation croisée sur le modèle XGB Regressor
scores = cross_val_score(xgb_regressor, X_train_concat, y_train, cv=5, scoring='r2')

# affichage des résultats
print("Scores de validation croisée du xgb Random Search (R²) :", scores)
print("Moyenne des scores de validation croisée du xgb Random Search (R²) :", np.mean(scores))

## c. Random Forest Regressor

In [ ]:
# Instanciation du modèle Random Forest Regressor et entrainement sur le train set
from sklearn.ensemble import RandomForestRegressor

rf_regressor = RandomForestRegressor(random_state=42)
rf_regressor.fit(X_train_concat, y_train)

In [ ]:
joblib.dump(rf_regressor, "model_rf_regressor")

In [ ]:
# Evaluation du modèle à l'aide de la méthode score sur les données d'entraînement et de test (coefficient de détermination ou R2)
print("Score R2 sur les données d'entraînement :", rf_regressor.score(X_train_concat, y_train))
print("Score R2 sur les données de test :", rf_regressor.score(X_test_concat, y_test))

In [ ]:
# Calcul des autres métriques sur les jeux d'entrainement et de test (MAE, MSE, RMSE)
y_pred_random_forest = rf_regressor.predict(X_test_concat)
y_pred_random_forest_train = rf_regressor.predict(X_train_concat)

# jeu d'entraînement
mae_random_forest_train = mean_absolute_error(y_train,y_pred_random_forest_train)
mse_random_forest_train = mean_squared_error(y_train,y_pred_random_forest_train,squared=True)
rmse_random_forest_train = mean_squared_error(y_train,y_pred_random_forest_train,squared=False)

# jeu de test
mae_random_forest_test = mean_absolute_error(y_test,y_pred_random_forest)
mse_random_forest_test = mean_squared_error(y_test,y_pred_random_forest,squared=True)
rmse_random_forest_test = mean_squared_error(y_test,y_pred_random_forest,squared=False)


# création d'un DataFrame
data_rf = {'MAE train': [mae_random_forest_train],
        'MAE test': [mae_random_forest_test],
        'MSE train': [mse_random_forest_train],
        'MSE test': [mse_random_forest_test],
        'RMSE train': [rmse_random_forest_train],
        'RMSE test': [rmse_random_forest_test]}

df_metrics_rf = pd.DataFrame(data_rf, index = ['Random Forest'])

df_metrics_rf.head()

In [ ]:
#Validation croisée sur le modèle Random Forest Regressor
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf_regressor, X_train_concat, y_train, cv=5, scoring='r2')
print("Scores de validation croisée (hyperparamètres par défaut): ", scores)
print("Moyenne des scores (hyperparamètres par défaut): ", scores.mean())

# *Les modèles les moins performants*

## d. Linear Regression

In [ ]:
# Instanciation du modèle Linear Regression

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
joblib.dump(lr, "model_lr")

In [ ]:
print('Score du modèle sur train:', lr.score(X_train, y_train))
print('Score du modèle sur test:', lr.score(X_test, y_test))

y_pred = lr.predict(X_test)
x_pred = lr.predict(X_train)

from sklearn.metrics import mean_squared_error, mean_squared_error, mean_absolute_error
mse_train_lr = mean_squared_error(y_train, x_pred)
mae_train_lr = mean_absolute_error(y_train, x_pred)
rmse_train_lr = mean_squared_error(y_train, x_pred, squared = False)

# affichage des métriques
print(f'MSE (train) : {mse_train_lr}')
print(f'MAE (train) : {mae_train_lr}')
print(f'RMSE (train) : {rmse_train_lr}')

mse_test_lr = mean_squared_error(y_test, y_pred)
mae_test_lr = mean_absolute_error(y_test, y_pred)
rmse_test_lr = mean_squared_error(y_test, y_pred, squared = False)

# affichage des métriques
print(f'MSE : {mse_test_lr}')
print(f'MAE : {mae_test_lr}')
print(f'RMSE : {rmse_test_lr}')

data = {"MSE train": mse_train_lr,
        "MAE train": mae_train_lr,
       "RMSE train": rmse_train_lr,
        "MSE test": mse_test_lr,
        "MAE test": mae_test_lr,
        "RMSE test": rmse_test_lr}

Tableau_lr = pd.DataFrame(data, index=['Regression Lineaire'])
display(Tableau_lr)

## e. Decision Tree Regressor

In [ ]:
# Instanciation du modèle Decision Tree Regressor

from sklearn.tree import DecisionTreeRegressor

regressor = DecisionTreeRegressor(random_state=42)

regressor.fit(X_train_concat, y_train)



In [ ]:
joblib.dump(regressor, "model_regressor")

In [ ]:
print(regressor.score(X_train_concat,y_train))
print(regressor.score(X_test_concat,y_test))

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# Prédictions
y_pred_regressor_train = regressor.predict(X_train_concat)
y_pred_regressor_test = regressor.predict(X_test_concat)


# Calcul des métriques pour DecisionTreeRegressor

# jeu d'entraînement
mae_regressor_train = mean_absolute_error(y_train, y_pred_regressor_train)
mse_regressor_train = mean_squared_error(y_train, y_pred_regressor_train, squared=True)
rmse_regressor_train = mean_squared_error(y_train, y_pred_regressor_train, squared=False)
# jeu de test
mae_regressor_test = mean_absolute_error(y_test, y_pred_regressor_test)
mse_regressor_test = mean_squared_error(y_test, y_pred_regressor_test, squared=True)
rmse_regressor_test = mean_squared_error(y_test, y_pred_regressor_test, squared=False)

# Création d'un DataFrame pour comparer les métriques des trois algorithmes
data_regressor_test = {
    'MAE train': [mae_regressor_train],
    'MAE test': [mae_regressor_test],
    'MSE train': [mse_regressor_train],
    'MSE test': [mse_regressor_test],
    'RMSE train': [rmse_regressor_train],
    'RMSE test': [rmse_regressor_test]
}

df_regressor_test = pd.DataFrame(data_regressor_test, index=['DecisionTreeRegressor'])

# Affichage du DataFrame
df_regressor_test.head()

## f. KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# Entraînement du modèle avec la meilleure valeur de n_neighbors
knn = KNeighborsRegressor(n_neighbors=3)
knn.fit(X_train, y_train)

In [ ]:
joblib.dump(knn, "model_knn")

In [ ]:
# Entraînement du modèle avec la distance de Minkowski (p=2)
knn_minkowski = KNeighborsRegressor(n_neighbors=3, metric='minkowski', p=2)
knn_minkowski.fit(X_train, y_train)
score_minkowski = knn_minkowski.score(X_test, y_test)

# Entraînement du modèle avec la distance de Manhattan
knn_manhattan = KNeighborsRegressor(n_neighbors=3, metric='manhattan')
knn_manhattan.fit(X_train, y_train)
score_manhattan = knn_manhattan.score(X_test, y_test)

# Entraînement du modèle avec la distance euclidienne (identique à Minkowski avec p=2)
knn_euclidienne = KNeighborsRegressor(n_neighbors=3, metric='euclidean')
knn_euclidienne.fit(X_train, y_train)
score_euclidienne = knn_euclidienne.score(X_test, y_test)

# Entraînement du modèle avec la distance de Tchebyshev
knn_tchebyshev = KNeighborsRegressor(n_neighbors=3, metric='chebyshev')
knn_tchebyshev.fit(X_train, y_train)
score_tchebyshev = knn_tchebyshev.score(X_test, y_test)


# Affichage des scores
print("Score Minkowski :",score_minkowski)
print("Score Manhattan :",score_manhattan)
print("Score Euclidienne :",score_euclidienne)
print("Score Tchebyshev :",score_tchebyshev)

In [ ]:
# Entraînement du modèle avec la distance de Mahalanobis
'''La distance de Mahalanobis nécessite la matrice de covariance'''
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
'''from sklearn.metrics import DistanceMetric # Plus besoin de DistanceMetric ici'''

V = np.cov(X_train.T)

# Définition d'une fonction pour calculer la distance de Mahalanobis
def mahalanobis_distance(x, y):
    return np.sqrt((x - y) @ np.linalg.inv(V) @ (x - y).T)

knn_mahalanobis = KNeighborsRegressor(n_neighbors=3, metric=mahalanobis_distance) # Utiliser la fonction personnalisée
knn_mahalanobis.fit(X_train, y_train)
score_mahalanobis = knn_mahalanobis.score(X_test, y_test)

# Affichage du score
print("Score Mahalanobis :", score_mahalanobis)

In [ ]:
# Entraînement du modèle avec la distance euclidienne standardisée (SEuclidienne)
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
Y_train_scaled = scaler.fit_transform(y_train.values.reshape(-1, 1))
Y_test_scaled = scaler.transform(y_test.values.reshape(-1, 1))

#Calcul de la variance V à partir de X_train_scaled
V = np.var(X_train_scaled, axis=0, ddof=1)


knn_seuclidienne = KNeighborsRegressor(n_neighbors=3, metric='seuclidean', metric_params={'V': V}) # Fournir V ici
knn_seuclidienne.fit(X_train_scaled, y_train)
score_seuclidienne = knn_seuclidienne.score(X_test_scaled, y_test)
print("Score SEuclidienne :", score_seuclidienne)

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# Évaluation du modèle sur le jeu d'entrainement
mse_train = mean_squared_error(Y_train_scaled, y_train)
mae_train = mean_absolute_error(Y_train_scaled, y_train)
rmse_train = mean_squared_error(Y_train_scaled, y_train, squared=False)

# Affichage des métriques
print(f'MSE (train) : {mse_train}')
print(f'MAE (train) : {mae_train}')
print(f'RMSE (train) : {rmse_train}')

# Évaluation du modèle sur le jeu de test
y_pred = knn_euclidienne.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred)
mae_test = mean_absolute_error(y_test, y_pred)
rmse_test = mean_squared_error(y_test, y_pred, squared=False)

# Affichage des métriques
print(f'MSE : {mse_test}')
print(f'MAE : {mae_test}')
print(f'RMSE : {rmse_test}')

data = {"MSE train": mse_train,
        "MAE train": mae_train,
       "RMSE train": rmse_train,
        "MSE test": mse_test,
        "MAE test": mae_test,
        "RMSE test": rmse_test}

df = pd.DataFrame(data, index=["KNN Euclidien"])
df

In [ ]:
# Évaluation du modèle à l'aide de la méthode score sur les données d'entraînement et de test (coefficient de détermination ou R2)
print("Score  sur les données d'entraînement :", knn_euclidienne.score(X_train, y_train))
print("Score  sur les données de test :", knn_euclidienne.score(X_test, y_test))

In [ ]:
# Entraînement du modèle avec la distance de Manhattan
knn_manhattan = KNeighborsRegressor(n_neighbors=3, metric='manhattan')
knn_manhattan.fit(X_train, y_train)
score_manhattan_train = knn_manhattan.score(X_train, y_train)
score_manhattan = knn_manhattan.score(X_test, y_test)

print("Score Manhattan sur les données d'entraînement :", score_manhattan_train)
print("Score Manhattan sur les données de test :", score_manhattan)

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# Évaluation du modèle sur le jeu d'entrainement
mse_train_mht = mean_squared_error(Y_train_scaled, y_train)
mae_train_mht = mean_absolute_error(Y_train_scaled, y_train)
rmse_train_mht = mean_squared_error(Y_train_scaled, y_train)

# Affichage des métriques
print(f'MSE (train) : {mse_train_mht}')
print(f'MAE (train) : {mae_train_mht}')
print(f'RMSE (train) : {rmse_train_mht}')

# Évaluation du modèle sur le jeu de test
y_pred = knn_manhattan.predict(X_test)
mse_test_mht = mean_squared_error(y_test, y_pred)
mae_test_mht = mean_absolute_error(y_test, y_pred)
rmse_test_mht = mean_squared_error(y_test, y_pred)

# Affichage des métriques
print(f'MSE : {mse_test_mht}')
print(f'MAE : {mae_test_mht}')
print(f'RMSE : {rmse_test_mht}')

data = {"MSE train": mse_train_mht,
        "MAE train": mae_train_mht,
       "RMSE train": rmse_train_mht,
        "MSE test": mse_test_mht,
        "MAE test": mae_test_mht,
        "RMSE test": rmse_test_mht}

tableau = pd.DataFrame(data, index=["KNN Manhattan"])
display(tableau)

estimator = knn
estimator.get_params()

# IV. Optimisation des modèles

# *Les modèles les plus performants (ordre décroissant)*

## a. Extra Trees Regressor

In [ ]:
# Affichage des hyperparamètres par défaut du modèle Extra Trees Regressor
estimator = et_regressor
estimator.get_params()

In [ ]:
# Recherche des hyperparamètres optimaux avec RandomizedSearchCV

from sklearn.model_selection import RandomizedSearchCV

# liste des hyperparamètres les plus importants
hyperparametres = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [None,5,10,15,20,25],
    'min_samples_split': [2,3,4, 5, 10, 15,20],
    'min_samples_leaf': [1, 2, 3, 4,6,8,10],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

# instanciation de RandomizedSearchCV
grid_et_regressor = RandomizedSearchCV(
    estimator=et_regressor,
    param_distributions=hyperparametres,
    n_iter=100,
    cv=5,
    n_jobs=-1,
    scoring='r2'
)

# exécutiton de la recherche
grid_et_regressor.fit(X_train_concat, y_train)

# évalutation des meilleurs paramètres sur l'ensemble d'entrainement
print("Meilleurs paramètres :", grid_et_regressor.best_params_)
print("Meilleur score :", grid_et_regressor.best_score_)

# évaluation du meilleur modèle sur l'ensemble de test
best_model = grid_et_regressor.best_estimator_
test_predictions = best_model.predict(X_test_concat)
test_score = r2_score(y_test, test_predictions)
print("Score R² sur l'ensemble de test :", test_score)

In [ ]:
# Rappel des meilleurs hyperparamètres de RandomizedSearchCV
best_hyperparams = {
    'n_estimators': 300,
    'max_depth': None,
    'min_samples_split': 5,
    'min_samples_leaf': 1,
    'criterion': 'squared_error',
    'bootstrap': False
}

# Instanciation d'un nouveau modèle avec les meilleurs hyperparamètres
optimized_et_regressor = ExtraTreesRegressor(
    n_estimators=best_hyperparams['n_estimators'],
    min_samples_split=best_hyperparams['min_samples_split'],
    min_samples_leaf=best_hyperparams['min_samples_leaf'],
    max_depth=best_hyperparams['max_depth'],
    criterion=best_hyperparams['criterion'],
    bootstrap=best_hyperparams['bootstrap'],
    random_state=42  # Assurer la reproductibilité
)

# Entraînement du modèle optimisé
optimized_et_regressor.fit(X_train_concat, y_train)

# Évaluation du modèle de base
base_model = ExtraTreesRegressor(random_state=42)
base_model.fit(X_train_concat, y_train)
base_train_score = base_model.score(X_train_concat, y_train)
base_test_score = base_model.score(X_test_concat, y_test)

print("Score R² sur l'ensemble d'entraînement (hyperparamètres par défaut) :", base_train_score)
print("Score R² sur l'ensemble de test (hyperparamètres par défaut) :", base_test_score)

print('----')

# Évaluation du modèle optimisé
optimized_et_regressor.fit(X_train_concat, y_train)
optimized_train_score = optimized_et_regressor.score(X_train_concat, y_train)
optimized_test_score = optimized_et_regressor.score(X_test_concat, y_test)

print("Score R² sur l'ensemble d'entraînement (meilleurs hyperparamètres RandomSearch) :", optimized_train_score)
print("Score R² sur l'ensemble de test (meilleurs hyperparamètres RandomSearch) :", optimized_test_score)



In [ ]:
#Calcul des autres métriques pour le modèle de base (MAE, MSE, RMSE)
base_train_predictions = base_model.predict(X_train_concat)
base_mae_train = mean_absolute_error(y_train, base_train_predictions)
base_mse_train = mean_squared_error(y_train, base_train_predictions)
base_rmse_train = mean_squared_error(y_train, base_train_predictions, squared=False)

base_test_predictions = base_model.predict(X_test_concat)
base_mae_test = mean_absolute_error(y_test, base_test_predictions)
base_mse_test = mean_squared_error(y_test, base_test_predictions)
base_rmse_test = mean_squared_error(y_test, base_test_predictions, squared=False)

# Calcul des autres métriques pour le modèle optimisé (MAE, MSE, RMSE)
optimized_train_predictions = optimized_et_regressor.predict(X_train_concat)
optimized_mae_train = mean_absolute_error(y_train, optimized_train_predictions)
optimized_mse_train = mean_squared_error(y_train, optimized_train_predictions)
optimized_rmse_train = mean_squared_error(y_train, optimized_train_predictions, squared=False)

optimized_test_predictions = optimized_et_regressor.predict(X_test_concat)
optimized_mae_test = mean_absolute_error(y_test, optimized_test_predictions)
optimized_mse_test = mean_squared_error(y_test, optimized_test_predictions)
optimized_rmse_test = mean_squared_error(y_test, optimized_test_predictions, squared=False)

#Création d'un DataFrame pour comparer les métriques des deux modèles
data = {
    'MAE train': [base_mae_train, optimized_mae_train],
    'MAE test': [base_mae_test, optimized_mae_test],
    'MSE train': [base_mse_train, optimized_mse_train],
    'MSE test': [base_mse_test, optimized_mse_test],
    'RMSE train': [base_rmse_train, optimized_rmse_train],
    'RMSE test': [base_rmse_test, optimized_rmse_test],
    'R² train': [base_train_score, optimized_train_score],
    'R² test': [base_test_score, optimized_test_score]
}

model_names = ['Hyperparamètres par défaut', 'Meilleurs hyperparamètres RandomSearch']

df_metrics_et_compare_random = pd.DataFrame(data, index=model_names)

display(df_metrics_et_compare_random)

In [ ]:
# Comparaison de la validation croisée sur le modèle non optmisé et optimisé
from sklearn.model_selection import cross_val_score


# Validation croisée sur le modèle de base
scores = cross_val_score(et_regressor, X_train_concat, y_train, cv=5, scoring='r2')
print("Scores de validation croisée (hyperparamètres par défaut): ", scores)
print("Moyenne des scores (hyperparamètres par défaut): ", scores.mean())
#print("Écart-type des scores R² (modèle de base): ", scores.std())

print('-----')

# Validation croisée sur le modèle optimisé
cv_scores = cross_val_score(optimized_et_regressor, X_train_concat, y_train, cv=5, scoring='r2')

print("Scores de validation croisée (meilleurs hyperparamètres RandomSearch):", cv_scores)
print("Moyenne des scores (meilleurs hyperparamètres RandomSearch):", cv_scores.mean())

## b. XGB Regressor

In [ ]:
# Identification des hyperparamètres (paramètres actuels du modèle)

estimator = xgb_regressor
estimator.get_params()

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Définition de la grille des hyperparamètres à tester
param_grid = {
    'n_estimators' : [100, 200, 300, 400, 500],
    'max_depth': [3, 5, 6, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample' : [0.5, 0.7, 1],
    'colsample_bytree' : [0.5, 0.7, 1],
    'gamma' : [0, 0.1, 0.2]
}

# Création d'une instance de XGBRegressor
xgb_regressor = XGBRegressor(objective='reg:squarederror', random_state=42)

# Configuration de GridSearchCV
grid_search = GridSearchCV(estimator=xgb_regressor, param_grid=param_grid, scoring='r2', cv=5)

# Entraînement du modèle
grid_search.fit(X_train_concat, y_train)

# Affichage des meilleurs paramètres
print("Meilleurs paramètres :", grid_search.best_params_)
print("Meilleur score de validation croisée (R²) :", grid_search.best_score_)

In [ ]:
# Création d'une instance de XGBRegressor
xgb_regressor2 = xgb.XGBRegressor(n_estimators=500, gamma = 0, subsample = 1, max_depth= 3, learning_rate= 0.2, colsample_bytree = 0.5, objective='reg:squarederror', random_state=42)

# Entraînement du modèle
xgb_regressor2.fit(X_train_concat, y_train)

print(xgb_regressor2.score(X_train_concat, y_train))
print(xgb_regressor2.score(X_test_concat, y_test))

In [ ]:
# Prédictions
y_pred_xgb_2_train = xgb_regressor2.predict(X_train_concat)
y_pred_xgb_2_test = xgb_regressor2.predict(X_test_concat)

# Calcul des scores
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_xgb_2_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_xgb_2_test))
train_r2 = r2_score(y_train, y_pred_xgb_2_train)
test_r2 = r2_score(y_test, y_pred_xgb_2_test)

# Calcul des métriques pour xgboost

# Jeu d'entraînement
mae_xgb2_train = mean_absolute_error(y_train, y_pred_xgb_2_train)
mse_xgb2_train = mean_squared_error(y_train, y_pred_xgb_2_train, squared=True)
rmse_xgb2_train = mean_squared_error(y_train, y_pred_xgb_2_train, squared=False)
# jeu de test
mae_xgb2_test = mean_absolute_error(y_test, y_pred_xgb_2_test)
mse_xgb2_test = mean_squared_error(y_test, y_pred_xgb_2_test, squared=True)
rmse_xgb2_test = mean_squared_error(y_test, y_pred_xgb_2_test, squared=False)

# Création d'un DataFrame pour comparer les métriques des trois algorithmes
data_xgb_test2 = {
    'MAE train': [mae_xgb2_train],
    'MAE test': [mae_xgb2_test],
    'MSE train': [mse_xgb2_train],
    'MSE test': [mse_xgb2_test],
    'RMSE train': [rmse_xgb2_train],
    'RMSE test': [rmse_xgb2_test]
}
# Création du DataFrame
df_xgb_test2 = pd.DataFrame(data_xgb_test2, index=['XGBoost'])

# Affichage du DataFrame
df_xgb_test2.head()


In [ ]:
# Création d'une instance de XGBRegressor
xgb_regressor2 = xgb.XGBRegressor(n_estimators=500, gamma = 0, subsample = 1, max_depth= 3, learning_rate= 0.2, colsample_bytree = 0.5, objective='reg:squarederror', random_state=42)

# Validation croisée sur le modèle XGBRegressor optimisé
scores2 = cross_val_score(xgb_regressor2, X_train_concat, y_train, cv=5, scoring='r2')

# Affichage des résultats
print("Scores de validation croisée du modèle de base (R²) :", scores2)
print("Moyenne des scores de validation croisée du modèle de base (R²) :", np.mean(scores2))

## c. Random Forest Regressor

In [ ]:
# Affichage des hyperparamètres par défaut du modèle Random Forest Regressor
estimator = rf_regressor
estimator.get_params()

In [ ]:
# Recherche des hyperparamètres optimaux avec RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score

hyperparametres_rf = {
    'criterion': ['squared_error'],
    'n_estimators': [50,60,70,80,90,100,150],
    'max_depth': [None, 40,50,60,80,100],
    'min_samples_split': [2,3,4,5],
    'min_samples_leaf': [1,2,3,4],
    'max_features': ['sqrt',5,10,15,18],
    'bootstrap': [False, True]
}

# Instanciation de RandomizedSearchCV
grid_rf_regressor = RandomizedSearchCV(
    estimator=rf_regressor,
    param_distributions=hyperparametres_rf,
    n_iter=100,
    cv=5,
    n_jobs=-1, 
    scoring='r2'
)

# Exécutiton de la recherche
grid_rf_regressor.fit(X_train_concat, y_train)


# Evalutation des meilleurs paramètres sur l'ensemble d'entrainement
print("Meilleurs paramètres :", grid_rf_regressor.best_params_)
print("Meilleur score :", grid_rf_regressor.best_score_)

# Evaluation du meilleur modèle sur l'ensemble de test
best_model = grid_rf_regressor.best_estimator_
test_predictions = best_model.predict(X_test_concat)
test_score = r2_score(y_test, test_predictions)
print("Score R² sur l'ensemble de test :", test_score)

In [ ]:
# Rappel des meilleurs hyperparamètres de RandomizedSearchCV
best_hyperparams = {
    'n_estimators': 68,
    'max_depth': 68,
    'min_samples_split': 2,
    'min_samples_leaf': 1,
    'max_features': 14,
    'criterion': 'squared_error',
    'bootstrap': False
}

# Instanciation d'un nouveau modèle avec les meilleurs hyperparamètres
optimized_rf_regressor = RandomForestRegressor(
    n_estimators=best_hyperparams['n_estimators'],
    min_samples_split=best_hyperparams['min_samples_split'],
    min_samples_leaf=best_hyperparams['min_samples_leaf'],
    max_features=best_hyperparams['max_features'],
    max_depth=best_hyperparams['max_depth'],
    criterion=best_hyperparams['criterion'],
    bootstrap=best_hyperparams['bootstrap'],
    random_state=42  # Assurer la reproductibilité
)

# Entraînement du modèle avec les meilleurs hyperparamètres
optimized_rf_regressor.fit(X_train_concat, y_train)

# Évaluation du modèle avec les hyperparamètres par défaut
base_model = RandomForestRegressor(random_state=42)
base_model.fit(X_train_concat, y_train)
base_train_score = base_model.score(X_train_concat, y_train)
base_test_score = base_model.score(X_test_concat, y_test)

print("Score R² sur l'ensemble d'entraînement (hyperparamètres par défaut) :", base_train_score)
print("Score R² sur l'ensemble de test (hyperparamètres par défaut) :", base_test_score)

print('----')

# Évaluation du modèle avec les meilleurs hyperparamètres
optimized_rf_regressor.fit(X_train_concat, y_train)
optimized_train_score = optimized_rf_regressor.score(X_train_concat, y_train)
optimized_test_score = optimized_rf_regressor.score(X_test_concat, y_test)

print("Score R² sur l'ensemble d'entraînement (meilleurs hyperparamètres RandomSearch) :", optimized_train_score)
print("Score R² sur l'ensemble de test (meilleurs hyperparamètres RandomSearch) :", optimized_test_score)



In [ ]:
# Calcul des autres métriques pour le modèle avec les hyperparamètres par défaut (MAE, MSE, RMSE)

from sklearn.metrics import mean_absolute_error, mean_squared_error

base_train_predictions = base_model.predict(X_train_concat)
base_mae_train = mean_absolute_error(y_train, base_train_predictions)
base_mse_train = mean_squared_error(y_train, base_train_predictions)
base_rmse_train = mean_squared_error(y_train, base_train_predictions, squared=False)

base_test_predictions = base_model.predict(X_test_concat)
base_mae_test = mean_absolute_error(y_test, base_test_predictions)
base_mse_test = mean_squared_error(y_test, base_test_predictions)
base_rmse_test = mean_squared_error(y_test, base_test_predictions, squared=False)

# Calcul des autres métriques pour le modèle avec les meilleurs hyperparamètres (MAE, MSE, RMSE)
optimized_train_predictions = optimized_rf_regressor.predict(X_train_concat)
optimized_mae_train = mean_absolute_error(y_train, optimized_train_predictions)
optimized_mse_train = mean_squared_error(y_train, optimized_train_predictions)
optimized_rmse_train = mean_squared_error(y_train, optimized_train_predictions, squared=False)

optimized_test_predictions = optimized_rf_regressor.predict(X_test_concat)
optimized_mae_test = mean_absolute_error(y_test, optimized_test_predictions)
optimized_mse_test = mean_squared_error(y_test, optimized_test_predictions)
optimized_rmse_test = mean_squared_error(y_test, optimized_test_predictions, squared=False)

#Création d'un DataFrame pour comparer les métriques des deux modèles
data = {
    'MAE train': [base_mae_train, optimized_mae_train],
    'MAE test': [base_mae_test, optimized_mae_test],
    'MSE train': [base_mse_train, optimized_mse_train],
    'MSE test': [base_mse_test, optimized_mse_test],
    'RMSE train': [base_rmse_train, optimized_rmse_train],
    'RMSE test': [base_rmse_test, optimized_rmse_test],
    'R² train': [base_train_score, optimized_train_score],
    'R² test': [base_test_score, optimized_test_score]
}

model_names = ['Hyperparamètres par défaut', 'Meilleurs hyperparamètres RandomSearch']

df_metrics_rf_compare = pd.DataFrame(data, index=model_names)

display(df_metrics_rf_compare)

In [ ]:
# Comparaison de la validation croisée avec hyperparamètres par défaut et meilleurs hyperparamètres RandomSearch
from sklearn.model_selection import cross_val_score

# Validation croisée sur le modèle avec hyperparamètres par défaut
scores = cross_val_score(rf_regressor, X_train_concat, y_train, cv=5, scoring='r2')
print("Scores de validation croisée (hyperparamètres par défaut): ", scores)
print("Moyenne des scores de validation croisée (hyperparamètres par défaut): ", scores.mean())
#print("Écart-type des scores (modèle de base): ", scores.std())

print('-----')

# Validation croisée sur le modèle avec meilleurs hyperparamètres RandomSearch
cv_scores = cross_val_score(optimized_rf_regressor, X_train_concat, y_train, cv=5, scoring='r2')
print("Scores de validation croisée (meilleurs hyperparamètres avec RandomSearch):", cv_scores)
print("Moyenne des scores de validation croisée (meilleurs hyperparamètres avec RandomSearch):", cv_scores.mean())
#print("Écart-type des scores (modèle optimisé) :", cv_scores.std())


# *Les modèles les moins performants*

## d. Linear Regression

In [ ]:
# Régression linéaire avec GridSearch

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.datasets import make_regression

# Génération des données de régression
X, y = make_regression(n_samples=100, n_features=2, noise=0.1)

# Définition du modèle
model = Ridge(solver="svd")

# Définition de la grille de paramètres
param_grid = {'alpha': [0.1, 1, 10, 100]}

# Configuration du GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)

# Ajustement du modèle
grid_search.fit(X, y)

# Affichage des meilleurs paramètres
print("Meilleurs paramètres : ", grid_search.best_params_)

# Affichage du score
print("Meilleur score : ", grid_search.best_score_)

In [ ]:
# Entrainement Regression lineaire avec Alpha = 0,1
model = Ridge(solver= "svd",alpha=0.1)
model.fit(X_train, y_train)
print('Score du modèle sur train:', model.score(X_train, y_train))
print('Score du modèle sur test:', model.score(X_test, y_test))

## e. Decision Tree Regressor

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Création d'une instance de l'arbre de régression
tree = DecisionTreeRegressor(random_state=42)

# Définition des valeurs possibles pour max_depth
param_grid = {'max_depth': range(1, 100)}

# Utilisation de GridSearchCV pour tester plusieurs profondeurs d'arbres avec un score de régression approprié
grid_search = GridSearchCV(estimator=tree, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Entraînement de GridSearchCV sur les données d'entraînement
grid_search.fit(X_train_concat, y_train)

# Meilleur paramètre max_depth
print(f"Meilleure profondeur d'arbre: {grid_search.best_params_['max_depth']}")

In [ ]:
from sklearn.tree import plot_tree # tree diagram

regressor2 = DecisionTreeRegressor(random_state=42, max_depth = 18)

regressor2.fit(X_train_concat, y_train)

print(regressor2.score(X_train_concat,y_train))
print(regressor2.score(X_test_concat,y_test))

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# Prédictions
y_pred_regressor2_train = regressor2.predict(X_train_concat)
y_pred_regressor2_test = regressor2.predict(X_test_concat)


# Calcul des métriques pour DecisionTreeRegressor

# jeu d'entraînement
mae_regressor2_train = mean_absolute_error(y_train, y_pred_regressor2_train)
mse_regressor2_train = mean_squared_error(y_train, y_pred_regressor2_train, squared=True)
rmse_regressor2_train = mean_squared_error(y_train, y_pred_regressor2_train, squared=False)

# jeu de test
mae_regressor2_test = mean_absolute_error(y_test, y_pred_regressor2_test)
mse_regressor2_test = mean_squared_error(y_test, y_pred_regressor2_test, squared=True)
rmse_regressor2_test = mean_squared_error(y_test, y_pred_regressor2_test, squared=False)

# Création d'un DataFrame pour comparer les métriques des trois algorithmes
data_regressor_test2 = {
    'MAE train': [mae_regressor2_train],
    'MAE test': [mae_regressor2_test],
    'MSE train': [mse_regressor2_train],
    'MSE test': [mse_regressor2_test],
    'RMSE train': [rmse_regressor2_train],
    'RMSE test': [rmse_regressor2_test]
}

df_regressor_test2 = pd.DataFrame(data_regressor_test2, index=['DecisionTreeRegressor'])

# Affichage du DataFrame
df_regressor_test2.head()

## f. KNN

In [ ]:
# Modification du modèle KNN avec de nouveaux hyperparamètres
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

# Définition de la grille de paramètres à tester
param_grid = {'n_neighbors': [3, 5, 7], 'metric': ['manhattan'] }

# Initialisation du modèle KNN
knn = KNeighborsRegressor()

# Initialisation du GridSearchCV
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5)

# Ajustement du modèle aux données d'entraînement
grid_search.fit(X_train, y_train)

# Affichage des meilleurs paramètres
print("Meilleurs paramètres : ", grid_search.best_params_)

# Affichage du score
print("Meilleur score : ", grid_search.best_score_)


In [ ]:
# Paramètres actuels du modèle
estimator = knn
estimator.get_params()

# V. Interprétation (Features Importances)

In [ ]:
''' Nous avons choisi les 3 modèles les plus performants pour faire notre Feature Importances (Extra Trees Regressor avec hyperparamètres par défaut, XGB Regressor avec hyperparamètres modifiés et Random Forest avec hyperparamètres modifiés)'''

## a. Extra Trees avec hyperparamètres par défaut

In [ ]:
# Récupération de l'importance des caractéristiques
importances = et_regressor.feature_importances_

# Création d'un DataFrame pour afficher les importances des caractéristiques
feature_importances = pd.DataFrame({
    'Feature': X_train_concat.columns,
    'Importance': et_regressor.feature_importances_
})

# Trier par importance décroissante
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Affichage des importances des caractéristiques
print(feature_importances)

# Création d'un DataFrame pour l'affichage
features = X_train_concat.columns
importance_df = pd.DataFrame({'Feature': features, 'Importance': importances})

# Trier par ordre décroissant
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Visualisation des importances des caractéristiques
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=importance_df, color='blue')
plt.title('Importances des Caractéristiques - Extra Trees Regressor')
plt.xlabel('Importance')
plt.ylabel('Caractéristiques')
plt.show()

## b. XGB Regressor avec hyperparamètres modifiés

In [ ]:
import matplotlib.pyplot as plt
import xgboost as xgb
import pandas as pd

# Création d'un DataFrame pour afficher les importances des caractéristiques
feature_importances = pd.DataFrame({
    'Feature': X_train_concat.columns,
    'Importance': xgb_regressor2.feature_importances_
})

# Trier par importance décroissante
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Affichage des importances des caractéristiques
print(feature_importances)

# Visualisation des importances des caractéristiques
plt.figure(figsize=(10, 8))
plt.barh(feature_importances['Feature'], feature_importances['Importance'], color='b')
plt.title('Importances des Caractéristiques - XGBRegressor')
plt.xlabel('Importance')
plt.ylabel('Caractéristiques')
plt.gca().invert_yaxis()  # Inverser l'ordre pour avoir les plus importantes en haut
plt.show()

## c. Random Forest avec hyperparamètres modifiés

In [ ]:
# Récupération de l'importance des caractéristiques
importances = optimized_rf_regressor.feature_importances_

# Création d'un DataFrame pour afficher les importances des caractéristiques
feature_importances = pd.DataFrame({
    'Feature': X_train_concat.columns,
    'Importance': optimized_rf_regressor.feature_importances_
})
# Trier par importance décroissante
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Affichage des importances des caractéristiques
print(feature_importances)

# Création d'un DataFrame pour l'affichage
features = X_train_concat.columns
importance_df = pd.DataFrame({'Feature': features, 'Importance': importances})

# Trier par ordre décroissant
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Visualisation des importances des caractéristiques
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=importance_df, color='blue')
plt.title('Importances des Caractéristiques - Random Forest Regressor')
plt.xlabel('Importance')
plt.ylabel('Caractéristiques')
plt.show()

# VI. Démonstration

In [ ]:
''' Afin d’évaluer concrètement la capacité de prédiction et de généralisation du modèle retenu, et de valider l’exactitude du classement du Features Importance, nous avons réalisé une démonstration finale avec les données de notre dataset puis avec de nouvelles données. '''

In [ ]:
# Calcul de la moyenne et de l'écart-type de X_train
mean_puissance = X_train['Puissance_fiscale'].mean()
std_puissance = X_train['Puissance_fiscale'].std()
mean_masse = X_train['Masse_OM_Max'].mean()
std_masse = X_train['Masse_OM_Max'].std()

In [ ]:
# Données pour les deux véhicules avec leurs caractéristiques
vehicule_A = {
    'Puissance_fiscale': 10,
    'Masse_OM_Max': 1915,
    'Energie_ESS+ELEC HNR': 0,
    'Energie_ESS+G.P.L.': 0,
    'Energie_ESSENCE': 0,
    'Energie_GAZ+ELEC HNR': 1,
    'Energie_GAZOLE': 0,
    'Energie_SUPERETHANOL': 0,
    'Carrosserie_BREAK': 0,
    'Carrosserie_CABRIOLET': 0,
    'Carrosserie_COMBISPACE': 0,
    'Carrosserie_COUPE': 0,
    'Carrosserie_MINIBUS': 0,
    'Carrosserie_MONOSPACE': 0,
    'Carrosserie_MONOSPACE COMPACT': 0,
    'Carrosserie_TS TERRAINS/CHEMINS': 1,
    'Type_de_boite_MECANIQUE': 0,
    'Type_de_boite_VARIATION CONTINUE': 0
}
vehicule_B = {
    'Puissance_fiscale': 8,
    'Masse_OM_Max': 1585,
    'Energie_ESS+ELEC HNR': 0,
    'Energie_ESS+G.P.L.': 0,
    'Energie_ESSENCE': 0,
    'Energie_GAZ+ELEC HNR': 0,
    'Energie_GAZOLE': 1,
    'Energie_SUPERETHANOL': 0,
    'Carrosserie_BREAK': 0,
    'Carrosserie_CABRIOLET': 0,
    'Carrosserie_COMBISPACE': 0,
    'Carrosserie_COUPE': 0,
    'Carrosserie_MINIBUS': 0,
    'Carrosserie_MONOSPACE': 0,
    'Carrosserie_MONOSPACE COMPACT': 0,
    'Carrosserie_TS TERRAINS/CHEMINS': 1,
    'Type_de_boite_MECANIQUE': 1,
    'Type_de_boite_VARIATION CONTINUE': 0
}

# Création du DataFrame
vehicules_df = pd.DataFrame([vehicule_A, vehicule_B])

# Calcul de la moyenne et de l'écart type pour les colonnes pertinentes
moyenne_puissance = X_train_concat['Puissance_fiscale'].mean()
ecart_type_puissance = X_train_concat['Puissance_fiscale'].std()
vehicules_df['Puissance_fiscale'] = (vehicules_df['Puissance_fiscale'] - mean_puissance) / std_puissance
vehicules_df['Masse_OM_Max'] = (vehicules_df['Masse_OM_Max'] - mean_masse) / std_masse

# Prédictions
predictions = optimized_et_regressor.predict(vehicules_df)
print(f"Prédiction pour le véhicule A : {predictions[0]}")
print(f"Prédiction pour le véhicule B : {predictions[1]}")